In [1]:
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.datasets import load_iris

In [2]:
iris= load_iris()

x=iris.data[:, (2,3)]
y= (iris.target==0).astype(np.int)

In [3]:
per_clf= Perceptron(random_state=42)
per_clf.fit(x,y)

c:\users\soumyama\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=42,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [4]:
y_pred= per_clf.predict([[2., 0.5]])

In [5]:
y_pred

array([1])

# DNN using TF on MNIST 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
n_inputs= 28*28
n_hidden1=300
n_hidden2=100
n_outputs=10

In [3]:
X= tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y= tf.placeholder(tf.int64, shape=(None), name="y")

In [4]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs= int(X.get_shape()[1])
        stddev= 2/np.sqrt(n_inputs)
        init= tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W= tf.Variable(init, name="weights")
        b= tf.Variable(tf.zeros([n_neurons]), name="biases")
        z= tf.matmul(X, W) + b
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

In [5]:
with tf.name_scope("dnn"):
    hidden1= neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2= neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits= neuron_layer(hidden2, n_outputs, "output")

In [6]:
from tensorflow.contrib.layers import fully_connected

with tf.name_scope("dnn"):
    hidden1= fully_connected(X, n_hidden1, scope="hidden1")
    hidden2= fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits= fully_connected(hidden2, n_outputs, scope="output", activation_fn=None)

In [7]:
with tf.name_scope("loss"):
    xentropy= tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss= tf.reduce_mean(xentropy, name="loss")

In [8]:
learning_rate= 0.01

with tf.name_scope("train"):
    optimizer= tf.train.GradientDescentOptimizer(learning_rate)
    training_op= optimizer.minimize(loss)

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy= tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
init= tf.global_variables_initializer()

In [11]:
saver= tf.train.Saver()

In [12]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train= X_train.reshape(-1, 784).astype("float32")/255.
X_test= X_test.reshape(-1, 784).astype("float32")/255.

y_train= y_train.astype("int32")
y_test= y_test.astype("int32")

In [13]:
n_epochs=400
batch_size=50

In [14]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [15]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train= accuracy.eval(feed_dict= {X: X_batch, y: y_batch})
        acc_test= accuracy.eval(feed_dict= {X: X_test, y: y_test})
        
        print("Epoch: ", epoch, "Train Accuracy: ", acc_train, "Test Accuracy: ", acc_test)
        
    save_path= saver.save(sess, "./checkpoint/MNIST_model.ckpt")

Epoch:  0 Train Accuracy:  0.86 Test Accuracy:  0.9016
Epoch:  1 Train Accuracy:  0.96 Test Accuracy:  0.9232
Epoch:  2 Train Accuracy:  0.92 Test Accuracy:  0.933
Epoch:  3 Train Accuracy:  0.96 Test Accuracy:  0.937
Epoch:  4 Train Accuracy:  0.92 Test Accuracy:  0.9436
Epoch:  5 Train Accuracy:  1.0 Test Accuracy:  0.948
Epoch:  6 Train Accuracy:  1.0 Test Accuracy:  0.9521
Epoch:  7 Train Accuracy:  0.96 Test Accuracy:  0.9549
Epoch:  8 Train Accuracy:  0.92 Test Accuracy:  0.9575
Epoch:  9 Train Accuracy:  1.0 Test Accuracy:  0.9593
Epoch:  10 Train Accuracy:  0.92 Test Accuracy:  0.9612
Epoch:  11 Train Accuracy:  0.98 Test Accuracy:  0.9639
Epoch:  12 Train Accuracy:  1.0 Test Accuracy:  0.9649
Epoch:  13 Train Accuracy:  0.96 Test Accuracy:  0.9662
Epoch:  14 Train Accuracy:  1.0 Test Accuracy:  0.967
Epoch:  15 Train Accuracy:  0.96 Test Accuracy:  0.9689
Epoch:  16 Train Accuracy:  1.0 Test Accuracy:  0.9694
Epoch:  17 Train Accuracy:  1.0 Test Accuracy:  0.9708
Epoch:  18 Tr

Epoch:  149 Train Accuracy:  1.0 Test Accuracy:  0.9808
Epoch:  150 Train Accuracy:  1.0 Test Accuracy:  0.9807
Epoch:  151 Train Accuracy:  1.0 Test Accuracy:  0.9807
Epoch:  152 Train Accuracy:  1.0 Test Accuracy:  0.9807
Epoch:  153 Train Accuracy:  1.0 Test Accuracy:  0.9811
Epoch:  154 Train Accuracy:  1.0 Test Accuracy:  0.9807
Epoch:  155 Train Accuracy:  1.0 Test Accuracy:  0.9807
Epoch:  156 Train Accuracy:  1.0 Test Accuracy:  0.9807
Epoch:  157 Train Accuracy:  1.0 Test Accuracy:  0.981
Epoch:  158 Train Accuracy:  1.0 Test Accuracy:  0.9809
Epoch:  159 Train Accuracy:  1.0 Test Accuracy:  0.9806
Epoch:  160 Train Accuracy:  1.0 Test Accuracy:  0.9809
Epoch:  161 Train Accuracy:  1.0 Test Accuracy:  0.9805
Epoch:  162 Train Accuracy:  1.0 Test Accuracy:  0.9808
Epoch:  163 Train Accuracy:  1.0 Test Accuracy:  0.9806
Epoch:  164 Train Accuracy:  1.0 Test Accuracy:  0.981
Epoch:  165 Train Accuracy:  1.0 Test Accuracy:  0.9807
Epoch:  166 Train Accuracy:  1.0 Test Accuracy:  0

Epoch:  296 Train Accuracy:  1.0 Test Accuracy:  0.9811
Epoch:  297 Train Accuracy:  1.0 Test Accuracy:  0.9808
Epoch:  298 Train Accuracy:  1.0 Test Accuracy:  0.9812
Epoch:  299 Train Accuracy:  1.0 Test Accuracy:  0.9808
Epoch:  300 Train Accuracy:  1.0 Test Accuracy:  0.9811
Epoch:  301 Train Accuracy:  1.0 Test Accuracy:  0.9808
Epoch:  302 Train Accuracy:  1.0 Test Accuracy:  0.981
Epoch:  303 Train Accuracy:  1.0 Test Accuracy:  0.9808
Epoch:  304 Train Accuracy:  1.0 Test Accuracy:  0.981
Epoch:  305 Train Accuracy:  1.0 Test Accuracy:  0.981
Epoch:  306 Train Accuracy:  1.0 Test Accuracy:  0.981
Epoch:  307 Train Accuracy:  1.0 Test Accuracy:  0.981
Epoch:  308 Train Accuracy:  1.0 Test Accuracy:  0.9812
Epoch:  309 Train Accuracy:  1.0 Test Accuracy:  0.9811
Epoch:  310 Train Accuracy:  1.0 Test Accuracy:  0.9809
Epoch:  311 Train Accuracy:  1.0 Test Accuracy:  0.9808
Epoch:  312 Train Accuracy:  1.0 Test Accuracy:  0.9809
Epoch:  313 Train Accuracy:  1.0 Test Accuracy:  0.98

In [20]:
with tf.Session() as sess:
    saver.restore(sess, "./checkpoint/MNIST_model.ckpt")
    X_new_scaled = X_test[619] # some new images (scaled from 0 to 1)
    Z = logits.eval(feed_dict={X: X_new_scaled.reshape(-1,784)})
    y_pred = np.argmax(Z, axis=1)
    print(y_pred)

INFO:tensorflow:Restoring parameters from ./checkpoint/MNIST_model.ckpt
[8]


### Ex 9

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

%matplotlib inline
tf.reset_default_graph()
tf.AUTO_REUSE 

os.chdir("C:/Users/soumyama/Documents/Python Scripts/Scikit Learn/")

In [2]:
(X_train, y_train), (X_test, y_test)= tf.keras.datasets.mnist.load_data()
X_train= X_train.reshape(-1, 28*28).astype(np.float32)/255.
X_test= X_test.reshape(-1, 28*28).astype(np.float32)/255.
y_train= y_train.astype(np.int32)
y_test= y_test.astype(np.int32)

X_valid, y_valid= X_train[:5000], y_train[:5000]
X_train, y_train= X_train[5000:], y_train[5000:]

In [3]:
n_inputs= 28*28
n_hidden1=300
n_hidden2=100
n_outputs=10
lr=0.01
n_epochs=1000
batch_size=50

In [4]:
X= tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y= tf.placeholder(tf.int32, shape=(None), name="y")

In [5]:
with tf.name_scope("dnn"):
    hidden1= tf.layers.dense(X, n_hidden1, activation= tf.nn.relu, name="hidden1")
    hidden2= tf.layers.dense(hidden1, n_hidden2, activation= tf.nn.relu, name="hidden2")
    logits= tf.layers.dense(hidden2, n_outputs, name="output")

In [6]:
with tf.name_scope("loss"):
    xentropy= tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss= tf.reduce_mean(xentropy, name="loss")
    loss_summary =tf.summary.scalar('log_loss', loss)

In [7]:
with tf.name_scope("train"):
    optimizer= tf.train.GradientDescentOptimizer(lr)
    train_op= optimizer.minimize(loss)

In [8]:
with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(logits, y, 1)
    accuracy= tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary= tf.summary.scalar('accuracy', accuracy)

In [9]:
init= tf.global_variables_initializer()
saver= tf.train.Saver()

In [10]:
def log_dir(prefix=""):
    now= datetime.utcnow().strftime("%Y%M%D%H%M%S")
    root_logdir= "C:/Users/soumyama/Documents/Python Scripts/Scikit Learn/tf_logs"
    if prefix:
        prefix +="-"
    name= prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [11]:
logdir= log_dir("MNIST_dnn")

In [12]:
# Write  tf logs

file_writer= tf.summary.FileWriter(logdir, tf.get_default_graph())

In [13]:
m,n= X_train.shape

In [14]:
n_batches= int(np.ceil(m / batch_size))

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

checkpoint_path= "C:/Users/soumyama/Documents/Python Scripts/Scikit Learn/checkpoint/deep_mnist_model.ckpt"
checkpoint_epoch_path= checkpoint_path + ".epoch"
final_model_path= "./deep_mnist_model/" 

In [15]:
best_loss= np.infty
epochs_without_progress= 0
max_epochs_without_progress=50

In [16]:
with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch= int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch=0
        sess.run(init)
        
    for epochs in range(start_epoch, n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(train_op, feed_dict= {X: X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str= sess.run([accuracy, 
                                                                                  loss, accuracy_summary, loss_summary], 
                                                                                 feed_dict= {X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epochs)
        file_writer.add_summary(loss_summary_str, epochs)
        if epochs % 5 == 0:
            print("Epoch: ", epochs, "\tValidation Accuracy: {:.3f}%".format(accuracy_val*100),
                 "\tValidation Loss: {:.3f}".format(loss_val))
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epochs+1))
            if loss_val < best_loss:
                saver.save(sess, final_model_path)
                best_loss= loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    print("Early Stopping")
                    break

Epoch:  0 	Validation Accuracy: 90.420% 	Validation Loss: 0.357
Epoch:  5 	Validation Accuracy: 95.100% 	Validation Loss: 0.180
Epoch:  10 	Validation Accuracy: 96.400% 	Validation Loss: 0.132
Epoch:  15 	Validation Accuracy: 97.000% 	Validation Loss: 0.109
Epoch:  20 	Validation Accuracy: 97.380% 	Validation Loss: 0.092
Epoch:  25 	Validation Accuracy: 97.640% 	Validation Loss: 0.085
Epoch:  30 	Validation Accuracy: 97.840% 	Validation Loss: 0.076
Epoch:  35 	Validation Accuracy: 98.020% 	Validation Loss: 0.072
Epoch:  40 	Validation Accuracy: 98.120% 	Validation Loss: 0.069
Epoch:  45 	Validation Accuracy: 98.180% 	Validation Loss: 0.068
Epoch:  50 	Validation Accuracy: 98.220% 	Validation Loss: 0.067
Epoch:  55 	Validation Accuracy: 98.140% 	Validation Loss: 0.067
Epoch:  60 	Validation Accuracy: 98.200% 	Validation Loss: 0.066
Epoch:  65 	Validation Accuracy: 98.220% 	Validation Loss: 0.067
Epoch:  70 	Validation Accuracy: 98.180% 	Validation Loss: 0.067
Epoch:  75 	Validation Accu

![png](https://user-images.githubusercontent.com/13174586/54521455-f9686f00-4990-11e9-9990-7ac4b69d1d7b.png)
